In [1]:
import numpy as np
import pandas as pd
from datetime import datetime
from sklearn import cross_validation

In [2]:
df = pd.read_csv("yellow_tripdata_2015-01-1p.csv")

In [3]:
df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RateCodeID,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
0,1,2015-01-29 09:00:36,2015-01-29 09:20:41,1,1.90,-74.005905,40.735645,1,N,-73.982353,40.749092,1,13.5,0.0,0.5,1.70,0.00,0.3,16.00
1,1,2015-01-16 21:59:15,2015-01-16 22:09:05,1,1.30,-73.988808,40.721855,1,N,-73.990845,40.734997,1,8.0,0.5,0.5,1.85,0.00,0.3,11.15
2,2,2015-01-09 19:29:35,2015-01-09 19:35:18,1,0.35,-73.980316,40.742496,1,N,-73.985909,40.745132,2,5.0,1.0,0.5,0.00,0.00,0.3,6.80
3,1,2015-01-28 14:28:17,2015-01-28 14:48:55,2,7.90,-73.873108,40.774086,1,N,-73.954384,40.781269,1,24.0,0.0,0.5,0.00,5.33,0.3,30.13
4,2,2015-01-22 07:08:12,2015-01-22 07:14:48,1,1.29,-73.961906,40.763901,1,N,-73.975761,40.753979,1,6.5,0.0,0.5,1.46,0.00,0.3,8.76


In [4]:
len(df)

127490

In [5]:
print df.dtypes

VendorID                   int64
tpep_pickup_datetime      object
tpep_dropoff_datetime     object
passenger_count            int64
trip_distance            float64
pickup_longitude         float64
pickup_latitude          float64
RateCodeID                 int64
store_and_fwd_flag        object
dropoff_longitude        float64
dropoff_latitude         float64
payment_type               int64
fare_amount              float64
extra                    float64
mta_tax                  float64
tip_amount               float64
tolls_amount             float64
improvement_surcharge    float64
total_amount             float64
dtype: object


In [6]:
date_object = datetime.strptime('2015-01-15 19:05:39', '%Y-%m-%d %H:%M:%S')
date_object

datetime.datetime(2015, 1, 15, 19, 5, 39)

In [7]:
def get_dt(x):
    return datetime.strptime(x, '%Y-%m-%d %H:%M:%S')

df['tpep_pickup_datetime'] = df.apply(lambda x: get_dt(x[1]), axis=1)
df['tpep_dropoff_datetime'] = df.apply(lambda x: get_dt(x[2]), axis=1)

df['tpep_pickup_dayofweek'] = df['tpep_pickup_datetime'].dt.dayofweek
#df['tpep_dropoff_dayofweek'] = df['tpep_dropoff_datetime'].dt.dayofweek

df['tpep_pickup_time'] = df['tpep_pickup_datetime'].dt.time
#df['tpep_dropoff_time'] = df['tpep_dropoff_datetime'].dt.time

df = df.drop(['store_and_fwd_flag', 'tpep_pickup_datetime', 'tpep_dropoff_datetime'], axis=1)

In [8]:
df.dtypes

VendorID                   int64
passenger_count            int64
trip_distance            float64
pickup_longitude         float64
pickup_latitude          float64
RateCodeID                 int64
dropoff_longitude        float64
dropoff_latitude         float64
payment_type               int64
fare_amount              float64
extra                    float64
mta_tax                  float64
tip_amount               float64
tolls_amount             float64
improvement_surcharge    float64
total_amount             float64
tpep_pickup_dayofweek      int64
tpep_pickup_time          object
dtype: object

In [9]:
df.head(10)

,VendorID,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RateCodeID,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,tpep_pickup_dayofweek,tpep_pickup_time
0,1,1,1.90,-74.005905,40.735645,1,-73.982353,40.749092,1,13.5,0.0,0.5,1.70,0.00,0.3,16.00,3,09:00:36
1,1,1,1.30,-73.988808,40.721855,1,-73.990845,40.734997,1,8.0,0.5,0.5,1.85,0.00,0.3,11.15,4,21:59:15
2,2,1,0.35,-73.980316,40.742496,1,-73.985909,40.745132,2,5.0,1.0,0.5,0.00,0.00,0.3,6.80,4,19:29:35
3,1,2,7.90,-73.873108,40.774086,1,-73.954384,40.781269,1,24.0,0.0,0.5,0.00,5.33,0.3,30.13,2,14:28:17
4,2,1,1.29,-73.961906,40.763901,1,-73.975761,40.753979,1,6.5,0.0,0.5,1.46,0.00,0.3,8.76,3,07:08:12
5,1,1,1.30,-73.959190,40.760860,1,-73.961464,40.770172,1,11.5,0.0,0.5,2.45,0.00,0.3,14.75,1,13:12:20
6,1,1,5.60,-73.870850,40.773743,1,-73.822807,40.727856,1,17.0,0.0,0.5,3.56,0.00,0.3,21.36,1,09:21:39
7,2,1,0.81,-73.952324,40.772816,1,-73.962959,40.775036,1,6.0,0.0,0.5,0.00,0.00,0.3,6.80,3,14:37:12
8,2,1,2.58,-73.998596,40.760700,1,-73.962769,40.758251,2,14.0,0.0,0.5,0.00,0.00,0.3,14.80,5,12:02:10
9,2,5,0.54,-73.963844,40.766808,1,-73.967735,40.762783,1,4.5,0.0,0.5,0.90,0.00,0.3,6.20,5,13:25:36


In [ ]:
# Set target variable name
target = 'tpep_dropoff_datetime'

# Set X and y
X = df.drop([target], axis=1)
y = df[target]


# Create separate training and test sets with 60/40 train/test split
X_train, X_test, y_train, y_test = cross_validation.train_test_split(X, y, test_size=0.4, random_state=0)


# check size of training set
print X_train.shape, y_train.shape

# check size of test set
print X_test.shape, y_test.shape

In [ ]:
X.head()

In [ ]:
y.head()